# CS3 Processing Report

This example showcases the use of Paramaterial to process and analyse a set of uniaxial compression test data.

In [ ]:
import paramaterial as pam
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
from paramaterial import DataSet, DataItem
print(pam.__version__)
print(np.__version__)
print(mpl.__version__)

## Data preparation

In [ ]:
pam.check_column_headers('data/01 raw data')
pam.check_for_duplicate_files('data/01 raw data')

Load the data and info into a dataset object (ds is shorthand for dataset).

In [ ]:
raw_ds = DataSet('info/01 raw info.xlsx', 'data/01 raw data').sort_by('temperature')

### Make the experimental matrix
We want to identify useful groupings and make visualisations.

The tests can be grouped by nominal rate, nominal temperature, and material. We will make the classic rate-temperature matrices - one for each material.

In [ ]:
cmap_gold = mpl.colors.LinearSegmentedColormap.from_list("", ["white", (85/255, 49/255, 0)])
mpl.rcParams['axes.facecolor'] = cmap_gold(0.1)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(3, 2.3))
heatmap_kwargs = dict(linewidths=2, cbar=False, annot=True, cmap=cmap_gold, vmax=9)

pam.experimental_matrix(raw_ds.subset({'material': 'AC'}).info_table,
                             index='temperature', columns='rate', as_heatmap=True,
                             title='AC', xlabel='Rate (/s)', ylabel='Temperature (°C)',
                             tick_params=dict(rotation=0), ax=axs[0], **heatmap_kwargs)

pam.experimental_matrix(raw_ds.subset({'material': 'H560'}).info_table,
                             index='temperature', columns='rate', as_heatmap=True,
                             title='H560', xlabel='Rate (/s)', ylabel=' ',
                             tick_params=dict(labelleft=False), ax=axs[1], **heatmap_kwargs)

pam.experimental_matrix(raw_ds.subset({'material': 'H580'}).info_table,
                             index='temperature', columns='rate', as_heatmap=True,
                             title='H580', xlabel='Rate (/s)', ylabel=' ',
                             tick_params=dict(labelleft=False), ax=axs[2], **heatmap_kwargs);

### Setup data overview plot
Now that we know how to group the data, we can set up a plotting function.

In [ ]:
styler = pam.plotting.Styler(color_by='temperature', color_by_label='(°C)', cmap='plasma').style_to(raw_ds)


def ds_subplot(ds: DataSet, **kwargs):
    return pam.plotting.dataset_subplots(
        ds, shape=(2, 3), xlim=(-0.2, 1.2), figsize=(8, 3.5),
        rows_by='rate', cols_by='material',
        row_vals=[[1], [10]], col_vals=[['AC'], ['H560'], ['H580']],
        col_titles=['AC', 'H560', 'H580'], row_titles=['1/s','10/s'],
        styler=styler, subplot_legend=False, plot_legend=False,
        **kwargs
    )


stress_strain_labels = dict(x='Strain', y='Stress(MPa)', xlabel='Strain (mm/mm)', ylabel='Stress (MPa)')

Plot the raw data.

In [ ]:
ds_subplot(raw_ds, **stress_strain_labels);

## Data processing

### Trim data

In [ ]:
def drop_columns(di: DataItem) -> DataItem:
    di.data = di.data.drop(columns=['PowAngle(deg)', 'Power(W)', 'Pram', 'PTemp', 'Stroke(mm)', 'wedge(mm)'])
    return di


def trim_using_time_step(di: DataItem) -> DataItem:
    t_diff = np.diff(di.data['Time(sec)'])
    di.data['time diff'] = np.hstack([t_diff[0], t_diff])
    di.data = di.data[di.data['time diff'] < 0.02][1:]
    return di


def remove_trailing_data(di: DataItem):
    di.data = di.data.iloc[:di.data['Force(kN)'].idxmax()]
    return di

In [ ]:
trimmed_ds = raw_ds.apply(drop_columns).apply(trim_using_time_step).apply(remove_trailing_data)
ds_subplot(trimmed_ds, **stress_strain_labels);

### Find flow stresses
Determine the stress, temperature for each test. Note that while nominal temperature is given, the actual test conditions are slightly different. The actual strain-rate does not differ significantly from the nominal rate for these tests.

In [ ]:
rates_ds = pam.calculate_strain_rate(trimmed_ds, time_key='Time(sec)')
processed_ds = pam.find_flow_stress_values(rates_ds, flow_strain=0.3,
                                           stress_key='Stress(MPa)', temperature_key='TC1(C)', rate_key='Strain_Rate')

processed_ds.write_output('info/02 processed info.xlsx', 'data/02 processed data')

### Make representative curves

In [ ]:
pam.make_representative_data(processed_ds, 'info/03 repr trim info.xlsx', 'data/03 repr trim data',
                             repres_col='Stress(MPa)', group_by_keys=['temperature', 'material', 'rate'],
                             interp_by='Strain', group_info_cols=['flow_Stress(MPa)'])
repr_ds = DataSet('info/03 repr trim info.xlsx', 'data/03 repr trim data', test_id_key='repres_id').sort_by('temperature')
ds_subplot(repr_ds, x='Strain', y='Stress(MPa)', xlabel='Strain', ylabel='Stress (MPa)',
           fill_between=('down_std_Stress(MPa)', 'up_std_Stress(MPa)'));

### Zener-Holloman Model

In [ ]:
info_table = processed_ds.info_table
info_table['Q_activation'] = 155000.
info_table['flow_temp(K)'] = processed_ds.info_table['flow_TC1(C)'] + 273.15
processed_ds.info_table = info_table

In [ ]:

analysis_ds = processed_ds.apply(pam.calculate_ZH_parameter, rate_key='flow_Strain_Rate',
                                 temperature_key='flow_temp(K)').sort_by(['material', 'rate'])

In [ ]:
from matplotlib.colors import ListedColormap
default_cmap = plt.cm.get_cmap('RdYlGn')
colors = default_cmap(np.arange(default_cmap.N))
colors[0] = (1, 1, 1, 1)
custom_cmap = ListedColormap(colors)


def conformance_matrix(_ds: DataSet, axs):
    heatmap_kwargs = dict(linewidths=4, cbar=False, annot=True, fmt='.1f', cmap=custom_cmap, vmin=80, vmax=100)
    pam.make_quality_matrix(_ds.subset({'material': 'AC'}).info_table, index='temperature', columns='rate',
                            flow_stress_key='flow_Stress(MPa)', as_heatmap=True, title='AC',
                            xlabel='Rate (/s)', ylabel='Temperature (°C)', tick_params=dict(rotation=0), ax=axs[0],
                            **heatmap_kwargs)
    pam.make_quality_matrix(_ds.subset({'material': 'H560'}).info_table, index='temperature', columns='rate',
                            flow_stress_key='flow_Stress(MPa)', as_heatmap=True, title='H560',
                            xlabel='Rate (/s)', ylabel=' ', tick_params=dict(labelleft=False), ax=axs[1],
                            **heatmap_kwargs)
    pam.make_quality_matrix(_ds.subset({'material': 'H580'}).info_table, index='temperature', columns='rate',
                            flow_stress_key='flow_Stress(MPa)', as_heatmap=True, title='H580',
                            xlabel='Rate (/s)', ylabel=' ', tick_params=dict(labelleft=False), ax=axs[2],
                            **heatmap_kwargs)
    return axs

In [ ]:
material_analysis_ds = pam.apply_ZH_regression(analysis_ds.copy(), flow_stress_key='flow_Stress(MPa)',
                                               group_by='material')
rate_analysis_ds = pam.apply_ZH_regression(analysis_ds, flow_stress_key='flow_Stress(MPa)', group_by='rate')

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(5, 4))
mat_cmap = mpl.colors.LinearSegmentedColormap.from_list("", mpl.cm.gnuplot(np.linspace(0, 1, 4))[:4])

pam.plot_ZH_regression(material_analysis_ds, group_by='material', color_by='material', cmap=mat_cmap, marker_by='rate',
                       flow_stress_key='flow_Stress(MPa)', rate_key='rate', temperature_key='flow_temp(K)', ax=axs,
                       eq_hscale=0.1);

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(3.5, 4))
conformance_matrix(material_analysis_ds, axs);

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(5, 4))
ax = pam.plot_ZH_regression(rate_analysis_ds, group_by='rate', color_by='rate', cmap='tab10', marker_by='material',
                            flow_stress_key='flow_Stress(MPa)', rate_key='rate', temperature_key='flow_temp(K)', ax=axs,
                            eq_hscale=0.1)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(3.5, 4))
conformance_matrix(rate_analysis_ds, axs);

### Tables

In [ ]:
df = pam.make_representative_info(material_analysis_ds, group_by_keys=['material', 'rate', 'temperature'],
                                  group_info_cols=['lnZ_fit_residual', 'lnZ', 'ZH_parameter', 'flow_TC1(C)',
                                                   'flow_Stress(MPa)'])
table = df[['material', 'rate', 'temperature', 'flow_Stress(MPa)', 'flow_TC1(C)', 'ZH_parameter', 'lnZ']].sort_values(['temperature', 'material'])

In [ ]:
table_s1 = table[table['rate'] == 1].drop(columns=['rate'])
for col in ['flow_Stress(MPa)', 'flow_TC1(C)', 'lnZ']:
    table_s1[col] = table_s1[col].apply(lambda x: f'{x:.3g}')
table_s1.style.to_latex('info/s1_table.tex')

In [ ]:
table_s10 = table[table['rate'] == 10].drop(columns=['rate'])
for col in ['flow_Stress(MPa)', 'flow_TC1(C)',  'lnZ']:
    table_s10[col] = table_s10[col].apply(lambda x: f'{x:.3g}')
table_s10.style.to_latex('info/s10_table.tex')